In [1]:
import pandas as pd


data_names = ["sim_4_10000.opt", "sim_4_100000.opt", "sim_8_10000.opt", "sim_8_100000.opt", "sim_20_10000.opt", "sim_20_100000.opt",
       "sim_40_10000.opt", "sim_40_100000.opt", "sim_100_10000.opt", "sim_100_100000.opt"]
names = ['sim_4_10000_pyrho', 'sim_4_100000_pyrho', 'sim_8_10000_pyrho', 'sim_8_100000_pyrho', 'sim_20_10000_pyrho', 'sim_20_100000_pyrho', 
        'sim_40_10000_pyrho', 'sim_40_100000_pyrho', 'sim_100_10000_pyrho', 'sim_100_100000_pyrho']
j = 0
for i in data_names:

    data = pd.read_csv(i, sep = '\t', header = 0)

    starting_point = 21700000
    ending_point = 45700000
    total_length = ending_point - starting_point    
    
    
    def convert_dict_to_fixed_windows(data, window_size):
        fixed_windows = []
        total_windows = ( total_length // window_size)

        for i in range(total_windows):
            window_start = starting_point + i * window_size
            window_end = window_start + window_size
            rec_in_window = [(rec*(pos_2 - pos_1)) if pos_2 < window_end and pos_1 > window_start else (rec*(window_end - pos_1)) if pos_1 < window_end and pos_2 > window_end and pos_1 > window_start else (rec*(pos_2 - window_start)) if pos_1 < window_start and pos_2 > window_start and pos_2 < window_end else (rec*(window_end - window_start)) if pos_1 < window_start and pos_2 > window_end else 0 for pos_1, pos_2, rec in zip(data['pos_1'], data['pos_2'], data['rec'])]

            if rec_in_window:
                avg_rec = sum(rec_in_window) / window_size
                fixed_windows.append((window_start, window_end, avg_rec))
            else:
                fixed_windows.append((window_start, window_end, 0))

        fixed_windows_df = pd.DataFrame(fixed_windows, columns=['start', 'end', 'rec'])     
        return fixed_windows_df


    window_size = 100000
    
    fixed_windows_df = convert_dict_to_fixed_windows(data, window_size)
    fixed_windows_df.to_csv(f'{names[j]}.txt', sep = '\t', index=False)
    j += 1